<a href="https://colab.research.google.com/github/sowji2604/ct_tasks/blob/main/IMAGE_RECOGNITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
!pip install tensorflow

In [96]:
!pip install numpy

In [97]:
!pip install streamlit

In [98]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img , ImageDataGenerator
from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense , Dropout , Flatten , Conv2D , MaxPooling2D

In [99]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhinavkrjha/dog-breed-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dog-breed-classification


In [100]:
dirs = os.listdir('/kaggle/input/dog-breed-classification')
for dir in dirs:
  files = list(os.listdir('/kaggle/input/dog-breed-classification/'+dir))
  print(dir + ' Folder has: ' + str(len(files))+'images')

Afgan Hound dog Folder has: 50images
Bulldog dog Folder has: 50images
Beagle dog Folder has: 50images
Borzoi dog Folder has: 50images
Bichon Frise dog Folder has: 50images
American Bulldog dog Folder has: 48images
Alaskan Malamute dog Folder has: 50images
Bocker dog Folder has: 50images
Affenhuahua dog Folder has: 50images
Bugg dog Folder has: 50images
Belgian Tervuren dog Folder has: 50images
Boxer dog Folder has: 50images
Auggie dog Folder has: 50images
Akita dog Folder has: 49images


In [101]:
base_dir = '/kaggle/input/dog-breed-classification'
img_size = 180
batch = 32

In [102]:
train_ds =tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_size,img_size),
    batch_size=batch
)

val_ds =tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_size,img_size),
    batch_size=batch
)

Found 697 files belonging to 14 classes.
Using 558 files for training.
Found 697 files belonging to 14 classes.
Using 139 files for validation.


In [103]:
dog_names = train_ds.class_names
dog_names

['Affenhuahua dog',
 'Afgan Hound dog',
 'Akita dog',
 'Alaskan Malamute dog',
 'American Bulldog dog',
 'Auggie dog',
 'Beagle dog',
 'Belgian Tervuren dog',
 'Bichon Frise dog',
 'Bocker dog',
 'Borzoi dog',
 'Boxer dog',
 'Bugg dog',
 'Bulldog dog']

In [104]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [105]:
data_augmentation = Sequential([
    layers.RandomFlip('horizontal',input_shape=(img_size,img_size,3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [106]:
model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10)
])

In [107]:
# ... (previous code) ...

# Assuming there are 120 breeds in the dataset
num_classes = 120

model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes) # Changed to num_classes
])

# ... (rest of the code) ...

In [108]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [109]:
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_10 (Sequential)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_7 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 180, 180, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 90, 90, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 90, 90, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 45, 45, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     3,965,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 120)            │        15,480 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,004,120 (15.27 MB)

 Trainable params: 4,004,120 (15.27 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_ds,validation_data=val_ds,epochs=15)

Epoch 1/15


In [ ]:
def classify_image(image_path):
  input_image = tf.keras.utils.load_img(image_path, target_size=(180,180))
  input_image_array = tf.keras.utils.img_to_array(input_image)
  input_image_exp_dim = tf.expand_dims(input_image_array,0)
  predictions = model.predict(input_image_exp_dim)
  result = tf.nn.softmax(predictions[0])
  outcome= 'the dog belong to '+ dog_names[np.argmax(result)] + ' with a score of '+str(max(result)*100)
  return outcome

In [ ]:
classify_image('/kaggle/input/dog-breed-classification/Afgan Hound dog/Image_1.jpg')

In [ ]:
import os

afghan_hound_dir = '/kaggle/input/dog-breed-classification/Afgan Hound dog'
if os.path.exists(afghan_hound_dir):
    print(os.listdir(afghan_hound_dir))
else:
    print(f"Directory not found: {afghan_hound_dir}")